# In-Silico Perturbation

### 1.1 Option A: Delete Single Gene and View both Gene and Cell Embedding Shifts

### 1.2 Option B: Delete all Genes and compare Cell State Shifts (Group A -> Group B)

### 1.3 Option C: Overexpress Gene

### 1.4 Option D: Utilize Anchor Token

### 1.5 Option E: Load in Cell Classifier for Perturbation